In [8]:
import psycopg2
import clickhouse_connect


# Подключение к PostgreSQL
def conn_pg():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="dbpg",
            user="user",
            password="password",
            port=5432
        )
        # print("Connected to PostgreSQL")
        # cur = conn.cursor()
        # cur.execute(query)
        # output = cur.fetchall()
        # print("PostgreSQL version:", db_version)
        # if output:
        #     return
        # else:
        #     print("No output from the query")
        return conn    
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
    
def conn_pg_close(conn):
    if conn:
        cur = conn.cursor()
        cur.close()
        conn.close()
        # print("PostgreSQL connection closed")


# Подключение к ClickHouse
client = clickhouse_connect.get_client(
            host='localhost'
            ,username='user'
            ,password='password'
            ,database='dbch'
            ,port=8123
            )
        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')
        # print("ClickHouse version:", version)

In [6]:
# Создание курсора
conn = conn_pg()

cur = conn.cursor()

# Создание таблицы
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary NUMERIC
);
''')
print("Table 'employees' created")

# Вставка данных в таблицу
cur.execute('''
INSERT INTO employees (name, position, salary)
VALUES ('Alice', 'Manager', 70000),
        ('Bob', 'Developer', 60000),
        ('Charlie', 'Designer', 50000);
''')
print("Data inserted into 'employees' table")

# Сохранение изменений
conn.commit()


Table 'employees' created
Data inserted into 'employees' table


In [7]:
# Закрытие курсора и соединения
conn_pg_close(conn)

In [9]:
try:
    # Создание таблицы
    client.command('''
    CREATE TABLE IF NOT EXISTS dbch.employees (
        id UInt32,
        name String,
        position String,
        salary Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
    ''')
    print("Table 'employees' created in ClickHouse")

except Exception as e:
    print(f"Error connecting to ClickHouse: {e}")


Table 'employees' created in ClickHouse


In [15]:
client.query('select * from dbch.employees')

In [16]:
# # Correct way to insert rows
# data = [
#     (1, 'Alice', 'Manager', 70000.0),
#     (2, 'Bob', 'Developer', 60000.0),
#     (3, 'Charlie', 'Designer', 50000.0)
# ]

# client.insert('dbch.employees', data, column_names=['id', 'name', 'position', 'salary'])